In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/hubble/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/hubble/')

In [2]:
from sql.MongoDBArticle import MongoDBArticle

In [3]:
# import requests

# r = requests.post('http://Carbine-env.eba-tgfee4hb.ap-south-1.elasticbeanstalk.com/update_candidates')

In [3]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [4]:
with PostgresDatabaseOperation() as cursor:
    sql = 'SELECT article_id FROM embeddings'
    cursor.execute(sql)
    results = cursor.fetchall()

In [5]:
embedding_articles = [x[0] for x in results]

In [6]:
with PostgresDatabaseOperation() as cursor:
    sql = 'SELECT article_id FROM llm_article_attributes'
    cursor.execute(sql)
    results = cursor.fetchall()    
attribute_articles = [x[0] for x in results]

In [7]:
from sql.MongoDBArticle import MongoDBArticle

In [8]:
def fetch_documents_after_date(min_published_date):
    query = {"is_premium_article": False, "published_time": {"$gt": min_published_date}}
    projection = {"_id": 1}
    collection = MongoDBArticle.get_collection()
    documents = list(collection.find(query, projection))
    return [str(doc['_id']) for doc in documents]

In [9]:
latest_mongo_articles = fetch_documents_after_date('2024-02-05')

In [10]:
len(latest_mongo_articles)

26330

In [11]:
missing_latest_embeddings = set(latest_mongo_articles) - set(embedding_articles)
missing_latest_attributes = set(latest_mongo_articles) - set(attribute_articles)

In [12]:
len(missing_latest_embeddings), len(missing_latest_attributes)

(2331, 3044)

In [13]:
union = list(set(missing_latest_attributes).union(set(missing_latest_embeddings)))

In [14]:
len(union)

3046

In [15]:
def create_missing_article_details(article_id):
    if article_id in missing_latest_embeddings:
        r = requests.post('http://Carbine-env.eba-tgfee4hb.ap-south-1.elasticbeanstalk.com/create_embedding', json={'articleId': article_id})
    if article_id in missing_latest_attributes:
        r = requests.post('http://Carbine-env.eba-tgfee4hb.ap-south-1.elasticbeanstalk.com/get_article_attributes_from_llm', json={'articleId': article_id})

In [2]:
import requests

In [12]:
r = requests.post('http://Camus-env.eba-metb4xtf.ap-south-1.elasticbeanstalk.com/get_article_attributes_from_llm', json={'articleId': '65dae59d1f6d9144de7edfd5'})

In [13]:
r.text

'{}'

In [16]:
chunk_size = 32
num_chunks = len(union)//chunk_size + 1

In [17]:
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
import requests

In [18]:
for i in range(num_chunks):
    article_chunk = union[i*chunk_size:(i+1)*chunk_size]
    print(f'starting creation at {"{:%b %d, %Y %H:%M}".format(datetime.now())}')
    start_time = time.time()
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = [executor.submit(create_missing_article_details, art_id) for art_id in article_chunk]

        for future in as_completed(futures):
            response = future.result()
            if response:  # Timeout or other errors
                completed_article_ids.append(response)
                continue
    print(f'completed chunk {(i + 1) * chunk_size} articles at {"{:%b %d, %Y %H:%M}".format(datetime.now())} in {int(time.time() - start_time)/60} minutes')

starting creation at Feb 14, 2024 14:21
completed chunk 32 articles at Feb 14, 2024 14:25 in 4.433333333333334 minutes
starting creation at Feb 14, 2024 14:25
completed chunk 64 articles at Feb 14, 2024 14:30 in 4.866666666666666 minutes
starting creation at Feb 14, 2024 14:30
completed chunk 96 articles at Feb 14, 2024 14:35 in 4.566666666666666 minutes
starting creation at Feb 14, 2024 14:35
completed chunk 128 articles at Feb 14, 2024 14:38 in 3.5166666666666666 minutes
starting creation at Feb 14, 2024 14:38
completed chunk 160 articles at Feb 14, 2024 14:42 in 3.5166666666666666 minutes
starting creation at Feb 14, 2024 14:42
completed chunk 192 articles at Feb 14, 2024 14:45 in 3.1 minutes
starting creation at Feb 14, 2024 14:45
completed chunk 224 articles at Feb 14, 2024 14:48 in 3.15 minutes
starting creation at Feb 14, 2024 14:48
completed chunk 256 articles at Feb 14, 2024 14:51 in 3.183333333333333 minutes
starting creation at Feb 14, 2024 14:51
completed chunk 288 articles